# Extending pgmpy

It's really easy to extend pgmpy to quickly prototype your ideas. pgmpy has a base abstract class for most of main functionalities like: `BaseInference` for inference, `BaseFactor` for model parameters, `BaseEstimators` for parameter and model learning. For adding a new feature to pgmpy we just need to implement a new class inheriting one of these base classes and then we can use the new class with other functionality of pgmpy.

In this example we will see how to write a new inference algorithm. We will take the example of a very simple algorithm in which we will multiply all the factors/CPD of the network and marginalize over variable to get the desired query.

pgmpy를 확장하여 아이디어를 신속하게 프로토타이핑하는 것은 정말 쉽습니다.

pgmpy에는 추론을 위한 `BaseInference`, 모델 매개변수를 위한 `BaseFactor`, 매개변수 및 모델 학습을 위한 `BaseEstimators`와 같은 대부분의 주요 기능에 대한 기본 추상 클래스가 있습니다.

pgmpy에 새 기능을 추가하려면 이러한 기본 클래스 중 하나를 상속하는 새 클래스를 구현하기만 하면 됩니다. 그러면 새 클래스를 pgmpy의 다른 기능과 함께 사용할 수 있습니다.

이 예에서 우리는 새로운 추론 알고리즘을 작성하는 방법을 볼 것입니다. 네트워크의 모든 요인/CPD를 곱하고 변수를 주변화하여 원하는 쿼리를 얻는 매우 간단한 알고리즘의 예를 들어보겠습니다.

In [1]:
# 간단한 정확한 추론 알고리즘

import itertools

from pgmpy.inference.base import Inference
from pgmpy.factors import factor_product


class SimpleInference(Inference):
    # 추론을 상속하여 클래스에서 self.model, self.factor 및 self.cardinality를 사용할 수 있습니다.
    def query(self, var, evidence):
        # self.factors는 {node: [factors_involving_node]} 형식의 사전입니다.
        factors_list = set(itertools.chain(*self.factors.values()))
        product = factor_product(*factors_list)
        reduced_prod = product.reduce(evidence, inplace=False)
        reduced_prod.normalize()
        var_to_marg = (
            set(self.model.nodes()) - set(var) - set([state[0] for state in evidence])
        )
        marg_prod = reduced_prod.marginalize(var_to_marg, inplace=False)
        return marg_prod

In [2]:
# 모델 정의

from pgmpy.models import BayesianModel
from pgmpy.factors.discrete import TabularCPD

model = BayesianModel([("A", "J"), ("R", "J"), ("J", "Q"), ("J", "L"), ("G", "L")])
cpd_a = TabularCPD("A", 2, values=[[0.2], [0.8]])
cpd_r = TabularCPD("R", 2, values=[[0.4], [0.6]])
cpd_j = TabularCPD(
    "J",
    2,
    values=[[0.9, 0.6, 0.7, 0.1], [0.1, 0.4, 0.3, 0.9]],
    evidence=["A", "R"],
    evidence_card=[2, 2],
)
cpd_q = TabularCPD(
    "Q", 2, values=[[0.9, 0.2], [0.1, 0.8]], evidence=["J"], evidence_card=[2]
)
cpd_l = TabularCPD(
    "L",
    2,
    values=[[0.9, 0.45, 0.8, 0.1], [0.1, 0.55, 0.2, 0.9]],
    evidence=["J", "G"],
    evidence_card=[2, 2],
)
cpd_g = TabularCPD("G", 2, values=[[0.6], [0.4]])

model.add_cpds(cpd_a, cpd_r, cpd_j, cpd_q, cpd_l, cpd_g)

In [3]:
# SimpleInference로 추론하기
infer = SimpleInference(model)
a = infer.query(var=["A"], evidence=[("J", 0), ("R", 1)])

In [4]:
print(a)

+-----+----------+
| A   |   phi(A) |
|-----+----------|
| A_0 |   0.6000 |
| A_1 |   0.4000 |
+-----+----------+


In [5]:
# 변수 제거 알고리즘을 사용한 결과 비교

from pgmpy.inference import VariableElimination

infer = VariableElimination(model)
result = infer.query(["A"], evidence={"J": 0, "R": 1})
print(result["A"])

+-----+----------+
| A   |   phi(A) |
|-----+----------|
| A_0 |   0.6000 |
| A_1 |   0.4000 |
+-----+----------+


Similarly we can also create new classes for Factor or CPDs and add them to networks and do inference over it or can write a new estimator class.

마찬가지로 Factor 또는 CPD에 대한 새 클래스를 생성하고 네트워크에 추가하고 이에 대한 추론을 수행하거나 새 추정기 클래스를 작성할 수도 있습니다.